In [24]:
import tensorflow as tf
import matplotlib.pyplot as plt
 
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3],tf.float32)

x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

loss = tf.reduce_mean(tf.square(linear_model - y))

optimizer = tf.train.GradientDescentOptimizer(0.01)

train = optimizer.minimize(loss)

x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1000):    
    sess.run(train,{x:x_train,y:y_train})
    if step == 10 :
        print(step,sess.run(W),sess.run(b))
    
# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

10 [-0.38310388] [-0.47581547]
W: [-0.9735152] b: [0.92213166] loss: 0.0010126778


# 数据流图（Data Flow Graph）
数据流图用“结点”（nodes）和“线”(edges)的有向图来描述数学计算。“节点” 一般用来表示施加的数学操作，但也可以表示数据输入（feed in）的起点/输出（push out）的终点，或者是读取/写入持久变量（persistent variable）的终点。“线”表示“节点”之间的输入/输出关系。这些数据“线”可以输运“size可动态调整”的多维数据数组，即“张量”（tensor）。张量从图中流过的直观图像是这个工具取名为“Tensorflow”的原因。一旦输入端的所有张量准备好，节点将被分配到各种计算设备完成异步并行地执行运算。一句话，TensorFlow基于数据流图，用于大规模分布式数值计算的开源框架，节点表示某种抽象的计算，边表示节点之间相互联系的张量。

![image.png](http://ata2-img.cn-hangzhou.img-pub.aliyun-inc.com/cdee2510cf622c4fabec8b48ae907794.png)

TensorFlow是用数据流图(data flow graphs)技术来进行数值计算的。数据流图是描述有向图中的数值计算过程。有向图中，节点通常代表数学运算，边表示节点之间的某种联系，它负责传输多维数据(Tensors)。节点可以被分配到多个计算设备上，可以异步和并行地执行操作。因为是有向图，所以只有等到之前的入度节点们的计算状态完成后，当前节点才能执行操作。

![image.png](http://ata2-img.cn-hangzhou.img-pub.aliyun-inc.com/7c88f2f33d5c79af9695e845d7736c25.png)

# TensorFlow编程模式

符号式编程 vs 命令式编程

和我们一般常用的命令式（Imperative）编程模式不同，TF采用的是符号式（Symbolic）编程。我们先认识一下两种编程模式：

命令式编程是很常见的编程模式，大多数Python或C++程序都采用命令式编程。命令式编程明确输入变量，根据程序逻辑逐步运算。下面是一段命令式编程的Python代码:

import numpy as np

a = np.ones(10)

b = np.ones(10) * 2

c = b * a

d = c + 1

执行完第一步a = np.ones(10)后，程序得到了输入变量a，第二句后得到了b，当执行c = b * a时，程序通过乘法计算而得到了c。

符号式编程则将计算过程抽象为计算图，所有输入节点、运算节点和输出节点均符号化处理。将上述命令式编程代码转换为符号式编程：

A = Variable('A')

B = Variable('B')

C = B * AD = C + Constant(1)

f = compile(D)

d = f(A=np.ones(10), B=np.ones(10)*2)

当前四步执行后，程序并没有A、B、C、D的值，仅仅是构建了由这四个符号构成的计算图，如下图所示：

大多数符号式编程的程序中都或显式或隐式地包含编译的步骤，将前面定义的计算图打包成可以调用的函数，而实际的计算则发生在编译后。

Theano和TensorFlow属于典型的符号式编程模式，而Torch则是命令式编程模式。在灵活性上，命令式编程模式更优，而在内存和计算上，符号式编程效率更高。

TensorFlow基本概念

要使用TensorFlow，我们必须理解TensorFlow：

使用图（Graph）表示计算流程

在会话（Session）中执行图

使用张量（Tensor）表示数据

使用变量（Variable）维护状态


# 张量 (tensor)

在 TensorFlow 中，数据不是以整数，浮点数或者字符串形式存在的，而是被封装在一个叫做 tensor 的对象中。Tensor是张量的意思，张量包含了0到任意维度的量，其中，0维的叫做常数，1维的叫做向量，二维叫做矩阵，多维度的就直接叫张量量。在 hello_constant = tf.constant(‘Hello World!’) 代码中，hello_constant是一个 0 维度的字符串 tensor，tensors 还有很多不同大小：
```
# tensor1 是一个0维的 int32 tensor
tensor1 = tf.constant(1234) 
# tensor2 是一个1维的 int32 tensor
tensor2 = tf.constant([123,456,789]) 
 # tensor3 是一个二维的 int32 tensor
tensor3 = tf.constant([ [123,456,789], [222,333,444] ])
```
2.Session是Tensorflow中的一个重要概念 
Tensorflow中的所有计算都构建在一张计算图中，这是一种对数学运算过程的可视化方法。就像刚才的代码：
```
with tf.Session() as sess:
    output = sess.run(hello_constant)
```
![image.png](http://ata2-img.cn-hangzhou.img-pub.aliyun-inc.com/ce24ad889919256dd0a004208a75e1fd.png)

这个session就是负责让这个图运算起来，session的主要任务就是负责分配GPU或者CPU的。
![image.png](http://ata2-img.cn-hangzhou.img-pub.aliyun-inc.com/3cdc5361d0400dbd2d39afc84f764bbb.png)
